In [1]:
# Deep Learning with PyTorch

#dir = '/home/liu/mysoft/pycharm/project/QuantistStudio/MyTest/test_pytorch/data/'

In [5]:

# Deep Learning Building Blocks: Affine maps, non-linearities and objectives

# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

lin = nn.Linear(5, 3)  # maps from R^5 to R^3, parameters A, b
# data is 2x5.  A maps from 5 to 3... can we map "data" under A?
data = autograd.Variable(torch.randn(2, 5))
print(lin(data))  # yes

torch.manual_seed(1)

Variable containing:
 0.0057  0.7352  0.2041
 1.2278 -0.2700  0.8397
[torch.FloatTensor of size 2x3]



In [6]:
# Non-Linearities

# In pytorch, most non-linearities are in torch.functional (we have it imported as F)
# Note that non-linearites typically don't have parameters like affine maps do.
# That is, they don't have weights that are updated during training.
data = autograd.Variable(torch.randn(2, 2))
print(data)
print(F.relu(data))

Variable containing:
-2.9718  1.7070
-0.4305 -2.2820
[torch.FloatTensor of size 2x2]

Variable containing:
 0.0000  1.7070
 0.0000  0.0000
[torch.FloatTensor of size 2x2]



In [7]:
# Softmax and Probabilities

# Softmax is also in torch.nn.functional
data = autograd.Variable(torch.randn(5))
print(data)
print(F.softmax(data))
print(F.softmax(data).sum())  # Sums to 1 because it is a distribution!
print(F.log_softmax(data))  # theres also log_softmax

Variable containing:
 0.5237
 0.0004
-1.2039
 3.5283
 0.4434
[torch.FloatTensor of size 5]

Variable containing:
 0.0437
 0.0259
 0.0078
 0.8823
 0.0403
[torch.FloatTensor of size 5]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
-3.1299
-3.6532
-4.8574
-0.1253
-3.2102
[torch.FloatTensor of size 5]



In [8]:
# Example: Logistic Regression Bag-of-Words classifier

data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

# word_to_ix maps each word in the vocab to a unique integer, which will be its
# index into the Bag of words vector
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2


class BoWClassifier(nn.Module):  # inheriting from nn.Module!

    def __init__(self, num_labels, vocab_size):
        # calls the init function of nn.Module.  Dont get confused by syntax,
        # just always do it in an nn.Module
        super(BoWClassifier, self).__init__()

        # Define the parameters that you will need.  In this case, we need A and b,
        # the parameters of the affine mapping.
        # Torch defines nn.Linear(), which provides the affine map.
        # Make sure you understand why the input dimension is vocab_size
        # and the output is num_labels!
        self.linear = nn.Linear(vocab_size, num_labels)

        # NOTE! The non-linearity log softmax does not have parameters! So we don't need
        # to worry about that here

    def forward(self, bow_vec):
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        # Many non-linearities and other functions are in torch.nn.functional
        return F.log_softmax(self.linear(bow_vec))


def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)


def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])


model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

# the model knows its parameters.  The first output below is A, the second is b.
# Whenever you assign a component to a class variable in the __init__ function
# of a module, which was done with the line
# self.linear = nn.Linear(...)
# Then through some Python magic from the Pytorch devs, your module
# (in this case, BoWClassifier) will store knowledge of the nn.Linear's parameters
for param in model.parameters():
    print(param)

# To run the model, pass in a BoW vector, but wrapped in an autograd.Variable
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
log_probs = model(autograd.Variable(bow_vector))
print(log_probs)

{'la': 4, 'is': 16, 'get': 20, 'at': 22, 'to': 8, 'en': 3, 'not': 17, 'Yo': 23, 'me': 0, 'cafeteria': 5, 'idea': 15, 'No': 9, 'comer': 2, 'que': 11, 'creo': 10, 'lost': 21, 'on': 25, 'good': 19, 'buena': 14, 'Give': 6, 'gusta': 1, 'una': 13, 'a': 18, 'si': 24, 'it': 7, 'sea': 12}
Parameter containing:

Columns 0 to 9 
-0.1599 -0.0406 -0.1231 -0.0440 -0.0606  0.0666 -0.0405  0.1708  0.0152  0.1358
-0.1411  0.1722 -0.1184  0.1092  0.1180  0.0847  0.1837  0.1188 -0.0732 -0.1597

Columns 10 to 19 
-0.0317 -0.0732  0.0726  0.0096 -0.1159 -0.0222  0.1483 -0.1061 -0.1854  0.0135
 0.0754  0.0071  0.1476  0.1432  0.1548  0.1291 -0.1628  0.1293 -0.1808 -0.0890

Columns 20 to 25 
 0.0669  0.1624 -0.0324 -0.0168  0.0230 -0.0272
-0.1295 -0.1729  0.1483  0.0669 -0.1575  0.0365
[torch.FloatTensor of size 2x26]

Parameter containing:
1.00000e-02 *
 -3.0944
  6.7328
[torch.FloatTensor of size 2]

Variable containing:
-1.0928 -0.4084
[torch.FloatTensor of size 1x2]



In [9]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

In [10]:
# Run on test data before we train, just to see a before-and-after
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

# Print the matrix column corresponding to "creo"
print(next(model.parameters())[:, word_to_ix["creo"]])

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Usually you want to pass over the training data several times.
# 100 is much bigger than on a real data set, but real datasets have more than
# two instances.  Usually, somewhere between 5 and 30 epochs is reasonable.
for epoch in range(100):
    for instance, label in data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Make our BOW vector and also we must wrap the target in a
        # Variable as an integer. For example, if the target is SPANISH, then
        # we wrap the integer 0. The loss function then knows that the 0th
        # element of the log probabilities is the log probability
        # corresponding to SPANISH
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        target = autograd.Variable(make_target(label, label_to_ix))

        # Step 3. Run our forward pass.
        log_probs = model(bow_vec)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

# Index corresponding to Spanish goes up, English goes down!
print(next(model.parameters())[:, word_to_ix["creo"]])

Variable containing:
-0.7920 -0.6032
[torch.FloatTensor of size 1x2]

Variable containing:
-0.4675 -0.9851
[torch.FloatTensor of size 1x2]

Variable containing:
1.00000e-02 *
 -3.1695
  7.5435
[torch.FloatTensor of size 2]



Variable containing:
-0.1186 -2.1903
[torch.FloatTensor of size 1x2]

Variable containing:
-2.1985 -0.1176
[torch.FloatTensor of size 1x2]

Variable containing:
 0.4547
-0.4110
[torch.FloatTensor of size 2]

